In [1]:
# PokerLogs and Timestamps 0.1.0
# 
# Author: Trevor Buteau
#
# The purpose of this program is to add accurrate timestamps to the PokerLogs
# output by the program PokerTH. The timestamps have previously been recorded
# by the "WatchDog" program, which has been watching the log folder for changes.
# Unfortunately, this does not detect every single change made to the logs (some
# times multiple changes are logged with the same time stamp), and part of the
# purpose of this script is to detect and fix these discrepancies in the timestamps.

import time
from datetime import datetime
import warnings

import numpy as np
from collections import deque

import pandas as pd

%matplotlib inline

In [10]:
#Reads a log file of time stamps that indicate when the PokerTH log file was modified (i.e. when an action was taken in the game).
def get_timestamps(ts_filename_):

    timestamps = []

    with open(ts_filename_) as f:
        for line in f:
            if "Modified file" in line:
                for word in line.split():
                    if "," in word:
                        timestamps.append(word)
                        
        # Lots of extra blank lines need to be added, in case of overflow/mismatch between timestamps and logs
        # particularly while debugging.
        for i in range(0,120):
            timestamps.append("00:00:00,000")
            
    return(timestamps)

In [11]:
# Reads a .txt log file exported by PokerTH, returns a pandas DataFrame with all game data for use later calculating things.
# Not all the time stamps will have been recorded equally, so the "time stamp buffer" 
# is used to adjust for this. Certain types of actions in the logs tend to be more likely
# to need time stamp adjustments, so most of these can be made automatically by this parsing
# script. A handful may have to be fixed manually, however.
def get_pokeractions(pokerlog_filename_,timestamps_,hero_name_,villain_name_):
    #Initializing the Data Frame
    blank_data = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0]).reshape((1,13))
    pokeractions = pd.DataFrame(blank_data, columns = ['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips', 'Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])

#     print(len(timestamps_))
    #Initializing the starting values for the data frame
    game_num = 1                    # Every time a game starts over
    hand_num = 0                    # Every time a new hand is dealt
    action_num = -1                 # 
    hand_start_index = action_num   # This is so that we can keep track of when we knew the hole cards
    shows_action_buffer = 0
    time_stamp_buffer = 0
    time_stamp = 0.0
    tie_detector = False
    small_blind = 10
    big_blind = 20
    hero_went_all_in = False
    villain_went_all_in = False
    hero_turn = False
    prev_show = False
    double_show = False
    game_phase = "STARTUP"
    game_end = False
    hero_chips = 5000
    villain_chips = 5000
    hole_cards = "[?s,?h]"
    board_cards = ""
    pot_chips = 0
    max_commit = 0
    hero_commit = 0
    villain_commit = 0
    hero_name = hero_name_
    villain_name = villain_name_
    dealer = villain_name
    log_line = ""
    

    foo = 0
    with open(pokerlog_filename_) as f:
        time_index = 0
        for line in f:
            log_line = line
            #Ignore the first line, and ignore blank lines
            if "Log-File for PokerTH" in line:
                pass
            elif line in ['\n', '\r\n']:
                pass
            
            else:
                if "-----------" in line: #This is the start of a new game, so many defaults should be reset.
                    hand_num += 1
                    words = line.split()
                    if words[5] == "1":
                        hero_chips = 5000
                        villain_chips = 5000
                        game_end = False
                    pot_chips = 0
                    hero_commit = 0
                    villain_commit = 0
                    max_commit = 0
                    hole_cards = "[?s,?h]"
                    board_cards = ""
                    game_phase = "PREFLOP"
                if "BLIND LEVEL" in line:
                    words = line.split()
                    small_blind = int(words[2][1:])
                    big_blind = int(words[4][1:])
                if "BLINDS" in line:
                    max_commit += big_blind
                    
                if villain_name + " starts as dealer" in line:
                    dealer = villain_name
                    if small_blind < villain_chips:
                        villain_chips = villain_chips - small_blind
                        villain_commit += small_blind
                    else:
                        villain_commit += villain_chips
                        villain_chips = 0
                    if big_blind < hero_chips:
                        hero_chips = hero_chips - big_blind
                        hero_commit += big_blind
                    else:
                        hero_commit += hero_chips
                        hero_chips = 0
                    
                    pot_chips = pot_chips + small_blind + big_blind
                if hero_name + " starts as dealer" in line:
                    dealer = hero_name
                    
                    if small_blind < hero_chips:
                        hero_chips = hero_chips - small_blind
                        hero_commit += small_blind
                    else:
                        hero_commit += hero_chips
                        hero_chips = 0
                    if big_blind < villain_chips:
                        villain_chips = villain_chips - big_blind
                        villain_commit += big_blind
                    else:
                        villain_commit += villain_chips
                        villain_chips = 0
                        
                    pot_chips = pot_chips + villain_commit + hero_commit

                if "Seat 1" in line:
                    pass
                    #print("Hero's seat...")
                if "Seat 2" in line:
                    pass
                    #print("Villain's seat...")
                if "PREFLOP" in line:
                    prev_show = False
                    double_show = False
                    tie_detector = False
                    hero_went_all_in = False
                    villain_went_all_in = False
                    if dealer == villain_name:
                        hero_turn = False
                    elif dealer == hero_name:
                        hero_turn = True
                    action_num += 1
                    hand_start_index = action_num
                    #Check the time difference between this action and the next
                    timestamp1 = timestamps_[action_num+time_stamp_buffer]
                    timestamp2 = timestamps_[action_num+time_stamp_buffer+1]
                    t1 = datetime.strptime(timestamp1, "%H:%M:%S,%f")
                    t2 = datetime.strptime(timestamp2, "%H:%M:%S,%f")
                    difference = t2-t1
                    #Sometimes PREFLOPs move through the system too quickly, and timestamps must be adjusted accordingly
                    if difference.seconds == 0 and difference.microseconds < 100000:
                        print("Found a PREFLOP timestamp difference of " + repr(difference) + " at Hand " +repr(hand_num))
                        print(t1)
                        print(t2)
                        time_stamp_buffer = time_stamp_buffer + 1
                    #Prepare the next row in the DF
                    time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips,pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips','Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                    
                if "FLOP [" in line:
                    game_phase = "FLOP"
                    if dealer == villain_name:
                        hero_turn = True
                    elif dealer == hero_name:
                        hero_turn = False
                    words = line.split()
                    board_cards = "[" + words[3]
                    action_num += 1
                    # As above with PREFLOP time differences.
                    timestamp1 = timestamps_[action_num+time_stamp_buffer-1]
                    timestamp2 = timestamps_[action_num+time_stamp_buffer]
                    t1 = datetime.strptime(timestamp1, "%H:%M:%S,%f")
                    t2 = datetime.strptime(timestamp2, "%H:%M:%S,%f")
                    difference = t2 - t1
                    if difference.seconds > 1:
                        time_stamp_buffer = time_stamp_buffer - 1
                        
                    #Prepping the next row in the dataframe
                    time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips, pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips','Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                    
                if "TURN" in line:
                    game_phase = "TURN"
                    if dealer == villain_name:
                        hero_turn = True
                    elif dealer == hero_name:
                        hero_turn = False
                    words = line.split()
                    board_cards = "[" + words[3]
                    action_num += 1
                    # As above with PREFLOP and FLOP, to detect a missed timestamp
                    timestamp1 = timestamps_[action_num+time_stamp_buffer-1]
                    timestamp2 = timestamps_[action_num+time_stamp_buffer]
                    t1 = datetime.strptime(timestamp1, "%H:%M:%S,%f")
                    t2 = datetime.strptime(timestamp2, "%H:%M:%S,%f")
                    difference = t2 - t1
                    if difference.seconds > 1:
                        time_stamp_buffer = time_stamp_buffer - 1
                        
                    #Prepping the next row in the DF
                    time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips, pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips','Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                    
                if "RIVER" in line:
                    game_phase = "RIVER"
                    if dealer == villain_name:
                        hero_turn = True
                    elif dealer == hero_name:
                        hero_turn = False
                    words = line.split()
                    board_cards = "[" + words[3]
                    action_num += 1
                    # As above w/PREFLOP, FLOP, and TURN
                    timestamp1 = timestamps_[action_num+time_stamp_buffer-1]
                    timestamp2 = timestamps_[action_num+time_stamp_buffer]
                    t1 = datetime.strptime(timestamp1, "%H:%M:%S,%f")
                    t2 = datetime.strptime(timestamp2, "%H:%M:%S,%f")
                    difference = t2 - t1
                    if difference.seconds > 1:
                        time_stamp_buffer = time_stamp_buffer - 1
                        
                    #Prepping the next frame in the DataFrame
                    time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips, pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips','Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                    
                if "checks" in line:
                    # No action, really, just update the dataframe.
                    action_num += 1

                    time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    hero_turn = not hero_turn
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips, pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips','Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                if "bets" in line:

                    action_num += 1
                    time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    hero_turn = not hero_turn
                    if villain_name in line:
                        words = line.split()
                        bet_size = int(words[2][1:-1])
                        villain_chips = villain_chips - bet_size
                        pot_chips += bet_size
                        villain_commit += bet_size
                        #Villain put hero all in
                        if hero_commit < villain_commit:
                            max_commit = villain_commit
                        else:
                            max_commit = hero_commit

                    if hero_name in line:
                        words = line.split()
                        bet_size = int(words[2][1:-1])
                        #print(bet_size)
                        hero_chips = hero_chips - bet_size
                        pot_chips += bet_size
                        #cost_to_stay = max_commit - bet_size
                        hero_commit += bet_size
                        if hero_commit > villain_commit:
                            max_commit = hero_commit
                        else:
                            max_commit = villain_commit

                    #Prep the next row of the data frame
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips, pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips','Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                if "all in" in line:
                    action_num += 1
                    timestamp1 = timestamps_[action_num+time_stamp_buffer]
                    timestamp2 = timestamps_[action_num+time_stamp_buffer+3]
                    t1 = datetime.strptime(timestamp1, "%H:%M:%S,%f")
                    t2 = datetime.strptime(timestamp2, "%H:%M:%S,%f")
                    difference = t2 - t1
                    
                    time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    hero_turn = not hero_turn
                    if villain_name in line:
                        villain_went_all_in = True
                        words = line.split()
                        if villain_chips > hero_chips and villain_commit >= hero_commit:
                            pot_chips += hero_chips
                            villain_commit += hero_chips
                            max_commit += hero_chips
                            villain_chips -= hero_chips
                        else:
                            pot_chips += villain_chips
                            villain_commit += villain_chips
                            max_commit += villain_chips
                            villain_chips = 0
                    if hero_name in line:
                        hero_went_all_in = True
                        words = line.split()
                        if hero_chips > villain_chips and hero_commit >= villain_commit:
                            pot_chips += villain_chips
                            hero_commit += villain_chips
                            max_commit += villain_chips
                            hero_chips -= villain_chips
                        else:
                            pot_chips += hero_chips
                            hero_commit += hero_chips
                            max_commit += hero_chips
                            hero_chips = 0
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips, pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips', 'Pot Chips','Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                    if difference.seconds > 0:
                        print("Made a timestamp adjustment BECAUSE WE'RE ALL IN")
                        print("At " + repr(game_phase) + " " +", Hand" + repr(hand_num))
                    
                if "calls" in line:
                    action_num += 1
                    time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    hero_turn = not hero_turn
                    if villain_name in line:
                        words = line.split()
                        cost_to_stay = hero_commit - villain_commit
                        pot_chips += cost_to_stay
                        villain_chips -= cost_to_stay
                        villain_commit = max_commit
                    if hero_name in line:
                        words = line.split()
                        cost_to_stay = villain_commit - hero_commit
                        pot_chips += cost_to_stay
                        hero_chips -= cost_to_stay
                        hero_commit = max_commit
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips, pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips','Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                if "shows" in line:   #Someone has just won, so 
                    shows_action_buffer += 1
                    # Checking to see if someone shows "unnecessarily", or shows after the chips have already been distributed
                    if pot_chips == 0 and villain_chips > 0 and hero_chips > 0: #and (game_phase != "RIVER"):  #SHOWING AFTER THE
                        shows_action_buffer = shows_action_buffer - 1
                        action_num += 1
                        time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    
                    if hero_name in line:
                        words = line.split()
                        if game_phase == "RIVER" and len(words) > 3 :
                            hole_cards = "[" + words[3]
                        else:
                            hole_cards = words[2]
                    #This is detecting if both were showing earlier – to detect if both players went all in
                    if prev_show == True:
                        shows_action_buffer += 1
                        prev_show = False
                    hero_turn = False
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips, pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips','Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                    pokeractions.loc[pokeractions['HandNum'] == str(hand_num), 'Hole Cards'] = hole_cards
                    prev_show = True
                if "wins" in line and "!" not in line:
                    if tie_detector == False:
                        action_num += 1
                        
                        # Fixing timestamp issues
                        timestamp1 = timestamps_[action_num+time_stamp_buffer]
                        timestamp2 = timestamps_[action_num+time_stamp_buffer+1]
                        timestamp3 = timestamps_[action_num+time_stamp_buffer-1]
                        t1 = datetime.strptime(timestamp1, "%H:%M:%S,%f")
                        t2 = datetime.strptime(timestamp2, "%H:%M:%S,%f")
                        t3 = datetime.strptime(timestamp3, "%H:%M:%S,%f")
                        difference = t2-t1
                        prev_diff = t1 - t3
                        if difference.seconds == 0 and difference.microseconds < 100000:
                            print("Found a 'wins' timestamp difference of " + repr(difference) + " at Hand " +repr(hand_num))
                            print(t1)
                            print(t2)
                            time_stamp_buffer = time_stamp_buffer + 1
                        if prev_diff.seconds > 1:
                            time_stamp_buffer = time_stamp_buffer - 1
                    time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    hero_turn = False
                    #The end of a game
                    if ("game " + str(game_num) + "!") in line:
                        print("GAME OVER")
                        pass
                    elif villain_name in line:
                        words = line.split()
                        #Give the chips to the villain
                        villain_chips += int(words[2][1:])
                        #Fix timestamp differences
                        if villain_went_all_in == True and difference.seconds == 0 and difference.microseconds < 100000:
                            print("moved it forward 1")
                            time_stamp_buffer = time_stamp_buffer +1
                        villain_went_all_in = False
                        pot_chips = 0
                    elif hero_name in line:
                        words = line.split()
                        #Fix timestamp differences
                        if hero_went_all_in == True and difference.seconds == 0 and difference.microseconds < 100000:
                            print("moved it forward 1")
                            time_stamp_buffer = time_stamp_buffer + 1
                        hero_went_all_in = False
                        #Give the chips to the Hero
                        hero_chips += int(words[2][1:])
                        pot_chips = 0
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips, pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips','Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                    tie_detector = True
                if "folds" in line:
                    action_num += 1
                    time_stamp = (timestamps_[action_num+time_stamp_buffer])
                    hero_turn = not hero_turn
                    next_action = np.array([hand_num,action_num,time_stamp,small_blind,big_blind, hero_turn,game_phase, hero_chips, villain_chips, pot_chips, hole_cards,board_cards,log_line]).reshape((1,13))
                    next_action_df = pd.DataFrame(next_action, columns=['HandNum','Action Num', 'TimeStamp','Small Blind', 'Big Blind', 'StartHeroTurn', 'Game Phase', 'Hero Chips', 'Villain Chips','Pot Chips', 'Hole Cards', 'Board Cards','Log Line'])
                    new_list = pokeractions, next_action_df
                    pokeractions = pd.concat(new_list)
                # Ignore this line, but it has probably been time-stamp-logged, so advance the time stamp buffer
                if "sits out" in line:
                    time_stamp_buffer += 1
                        
    print(action_num+time_stamp_buffer)
    return(pokeractions)

In [18]:
subject_map = pd.read_csv('subject_map.csv')
subject_num = 1
time_sub_num = 1
ts_filename = str(time_sub_num) + "_timestamps.log"
pokerlog_filename = str(subject_num) + "_pokerth-log.txt"
times = get_timestamps(ts_filename)
poker_acts = get_pokeractions(pokerlog_filename, times, subject_map.iloc[subject_num-1,2], subject_map.iloc[subject_num-1,3])
poker_acts = poker_acts.iloc[1:]
poker_acts = poker_acts.set_index('Action Num')
poker_acts.iloc[-1,1] = poker_acts.iloc[-2,1]

poker_acts

1269
Made a timestamp adjustment BECAUSE WE'RE ALL IN
At 'TURN' , Hand38
Found a PREFLOP timestamp difference of datetime.timedelta(0, 0, 11000) at Hand 43
1900-01-01 13:32:12.308000
1900-01-01 13:32:12.319000
Found a PREFLOP timestamp difference of datetime.timedelta(0, 0, 11000) at Hand 56
1900-01-01 13:37:49.060000
1900-01-01 13:37:49.071000
Found a PREFLOP timestamp difference of datetime.timedelta(0, 0, 10000) at Hand 64
1900-01-01 13:41:45.019000
1900-01-01 13:41:45.029000
Made a timestamp adjustment BECAUSE WE'RE ALL IN
At 'TURN' , Hand71
Found a PREFLOP timestamp difference of datetime.timedelta(0, 0, 10000) at Hand 75
1900-01-01 13:46:56.875000
1900-01-01 13:46:56.885000
Made a timestamp adjustment BECAUSE WE'RE ALL IN
At 'RIVER' , Hand95
1148


,HandNum,TimeStamp,Small Blind,Big Blind,StartHeroTurn,Game Phase,Hero Chips,Villain Chips,Pot Chips,Hole Cards,Board Cards,Log Line
Action Num,,,,,,,,,,,,
0,1,"13:10:45,920",10,20,True,PREFLOP,4990,4980,30,"[?s,?h]",,PREFLOP\n
1,1,"13:11:02,067",10,20,False,PREFLOP,4980,4980,40,"[?s,?h]",,Guest15166 calls $10.\n
2,1,"13:11:02,119",10,20,True,PREFLOP,4980,4960,60,"[?s,?h]",,Guest84440 bets $20.\n
3,1,"13:11:05,412",10,20,False,PREFLOP,4960,4960,80,"[?s,?h]",,Guest15166 calls $20.\n
4,1,"13:11:05,460",10,20,False,FLOP,4960,4960,80,"[?s,?h]","[2h,Ah,Ts]","FLOP [board cards 2h,Ah,Ts]\n"
5,1,"13:11:14,677",10,20,True,FLOP,4960,4960,80,"[?s,?h]","[2h,Ah,Ts]",Guest84440 checks.\n
6,1,"13:11:14,860",10,20,False,FLOP,4960,4960,80,"[?s,?h]","[2h,Ah,Ts]",Guest15166 checks.\n
7,1,"13:11:14,860",10,20,False,TURN,4960,4960,80,"[?s,?h]","[2h,Ah,Ts,Ad]","TURN [board cards 2h,Ah,Ts,Ad]\n"
8,1,"13:11:22,461",10,20,True,TURN,4960,4960,80,"[?s,?h]","[2h,Ah,Ts,Ad]",Guest84440 checks.\n


In [15]:
# To save output to .csv file
# poker_acts.to_csv(str(subject_num) + "_pokeracts_withcorrecttimestamps.csv")